# **Imports**

In [1]:
import pandas as pd

# **Reading Data**

In [2]:
df = pd.read_csv('../Data/combined_data.csv')

# **Data Exploration and Analysis**

In [3]:
df.head()

,Latitude,Longitude,Altitude,Distance,Alignment,Distance_Sensors,Proximity,Wheel_Speed,Velocity,Hall_Effect,Emergency_Stop,Temperature,Humidity,Light_Level
0,37.852811,-121.572439,52.355375,0.0000,False,3.525967,True,3.185937,0.955781,False,False,19.515751,69.466546,147.747689
1,37.852811,-121.572438,52.355375,0.0003,True,4.756672,True,1.620925,0.486278,False,False,24.445977,46.264885,129.928270
2,37.852811,-121.572437,52.355375,0.0006,False,0.712118,True,3.550108,1.065032,True,True,12.375221,52.270249,192.439571
3,37.852811,-121.572436,52.355375,0.0009,True,2.224535,True,4.835126,1.450538,True,False,21.161813,66.718895,441.660815
4,37.852811,-121.572435,52.355375,0.0012,False,1.514108,True,3.214580,0.964374,True,False,23.299301,35.738891,681.076538


In [4]:
df.shape

(50000, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Latitude          50000 non-null  float64
 1   Longitude         50000 non-null  float64
 2   Altitude          50000 non-null  float64
 3   Distance          50000 non-null  float64
 4   Alignment         50000 non-null  bool   
 5   Distance_Sensors  50000 non-null  float64
 6   Proximity         50000 non-null  bool   
 7   Wheel_Speed       50000 non-null  float64
 8   Velocity          50000 non-null  float64
 9   Hall_Effect       50000 non-null  bool   
 10  Emergency_Stop    50000 non-null  bool   
 11  Temperature       50000 non-null  float64
 12  Humidity          50000 non-null  float64
 13  Light_Level       50000 non-null  float64
dtypes: bool(4), float64(10)
memory usage: 4.0 MB


In [6]:
df.describe()

,Latitude,Longitude,Altitude,Distance,Distance_Sensors,Wheel_Speed,Velocity,Temperature,Humidity,Light_Level
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,37.418966,-121.597137,52.356226,0.149850,2.748821,2.749783,0.824935,22.472888,49.971703,498.657612
std,0.298670,0.310063,27.750864,0.086603,1.299671,1.300411,0.390123,7.242958,11.571847,288.051266
min,37.007134,-121.998704,0.814268,0.000000,0.500130,0.500098,0.150029,10.000107,30.001439,0.011402
25%,37.121428,-121.866625,28.797280,0.074925,1.621144,1.623092,0.486928,16.244802,39.959915,246.886811
50%,37.415549,-121.657832,52.170024,0.149850,2.743305,2.751643,0.825493,22.403462,49.968997,500.836109
75%,37.656867,-121.282915,79.472374,0.224775,3.871313,3.878636,1.163591,28.800494,60.021020,746.956417
max,37.992811,-121.001388,98.772346,0.299700,4.999977,4.999840,1.499952,34.999474,69.999612,999.987439


In [7]:
df.isna().sum()

Latitude            0
Longitude           0
Altitude            0
Distance            0
Alignment           0
Distance_Sensors    0
Proximity           0
Wheel_Speed         0
Velocity            0
Hall_Effect         0
Emergency_Stop      0
Temperature         0
Humidity            0
Light_Level         0
dtype: int64

In [8]:
df.columns

Index(['Latitude', 'Longitude', 'Altitude', 'Distance', 'Alignment',
       'Distance_Sensors', 'Proximity', 'Wheel_Speed', 'Velocity',
       'Hall_Effect', 'Emergency_Stop', 'Temperature', 'Humidity',
       'Light_Level'],
      dtype='object')

In [9]:
alignment_conditions = [
    alignment == True for alignment in df["Alignment"]
]

# Adding labels for Chaining Alignment
df["Alignment_Label"] = ["Aligned" if condition else "Not_Aligned" for condition in alignment_conditions]


In [10]:
# Define conditions for safety based on Hall_Effect and Emergency_Stop
safety_conditions = [
    hall_effect == True and emergency_stop == False
    for hall_effect, emergency_stop in zip(
        df["Hall_Effect"],
        df["Emergency_Stop"]
    )
]

# Adding labels for Safety Decision
df["Safety_Label"] = ["Safe" if condition else "Unsafe" for condition in safety_conditions]

In [11]:
# Speed thresholds (in meters per second - m/s)
threshold_high_speed = 1.5  

# Distance thresholds (in meter)
threshold_low_distance = 0.5 

# Temperature thresholds (in degrees Celsius)
threshold_high_temperature = 30 

# Light level thresholds (in Lux)
threshold_low_light = 100


collision_risk_levels = []

for distance, proximity, wheel_speed, temperature, light_level in zip(
    df["Distance"],
    df["Proximity"],
    df["Wheel_Speed"],
    df["Temperature"],
    df["Light_Level"]
):
    if proximity or wheel_speed > threshold_high_speed:
        collision_risk_levels.append(2)
    elif distance < threshold_low_distance:
        collision_risk_levels.append(1)
    elif temperature > threshold_high_temperature or light_level < threshold_low_light:
        collision_risk_levels.append(1)
    else:
        collision_risk_levels.append(0)


# Adding labels for Collision Avoidance
df["Collision_Avoidance_Label"] = ["Low_Risk" if risk_level == 0 else "Medium_Risk" if risk_level == 1 else "High_Risk" for risk_level in collision_risk_levels]

In [12]:
# Define acceptable distance thresholds for chaining 
max_distance = 5 
min_distance = 0.5

# Cart wheel speed treshold in meters per second (m/s) for chaining 
high_speed = 3.5

# Define conditions for chaining based on the data
chaining_conditions = [
    alignment and distance > min_distance and distance < max_distance and proximity and wheel_speed < high_speed and safety_label == "Safe"
    for alignment, distance, proximity, safety_label, wheel_speed in zip(
        df["Alignment"],
        df["Distance_Sensors"],
        df["Proximity"],
        df["Safety_Label"],
        df['Wheel_Speed']
    )
]

# Adding labels for Chaining/Unchaining Decision
df["Chaining_Unchaining_Label"] = ["Chaining" if condition else "Unchaining" for condition in chaining_conditions]

In [13]:
# Save the updated combined data with labels to a new CSV file
df.to_csv("../Data/final_combined_data.csv", index=False)

print("Combined data with labels saved.")

Combined data with labels saved.


In [14]:
df.columns

Index(['Latitude', 'Longitude', 'Altitude', 'Distance', 'Alignment',
       'Distance_Sensors', 'Proximity', 'Wheel_Speed', 'Velocity',
       'Hall_Effect', 'Emergency_Stop', 'Temperature', 'Humidity',
       'Light_Level', 'Alignment_Label', 'Safety_Label',
       'Collision_Avoidance_Label', 'Chaining_Unchaining_Label'],
      dtype='object')

In [15]:
df.isna().sum()

Latitude                     0
Longitude                    0
Altitude                     0
Distance                     0
Alignment                    0
Distance_Sensors             0
Proximity                    0
Wheel_Speed                  0
Velocity                     0
Hall_Effect                  0
Emergency_Stop               0
Temperature                  0
Humidity                     0
Light_Level                  0
Alignment_Label              0
Safety_Label                 0
Collision_Avoidance_Label    0
Chaining_Unchaining_Label    0
dtype: int64